# N gram language modeling
P(wi|wi−1,wi−2,…,wi−n+1) .. give n previous word, predict the probablity of the next word

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # context_size * embedding dimension goes as 1D array to 128 first layer Wx+b
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        # input R 128 -> R vocab size
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        # convert the input to 1D,
        # -1 dimension is inferred from other dimension
        embeds = self.embeddings(inputs).view((1, -1))
        # non linearity - RELU(WX+b)
        out = F.relu(self.linear1(embeds))
        # torch size [1, 20], 
        # print(embeds.size(), out.size())
        # exit()
        # second layer -> W out + b
        out = self.linear2(out)
        # calculate softmax
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[
 520.6215
[torch.FloatTensor of size 1]
, 
 517.9293
[torch.FloatTensor of size 1]
, 
 515.2582
[torch.FloatTensor of size 1]
, 
 512.6068
[torch.FloatTensor of size 1]
, 
 509.9751
[torch.FloatTensor of size 1]
, 
 507.3612
[torch.FloatTensor of size 1]
, 
 504.7645
[torch.FloatTensor of size 1]
, 
 502.1830
[torch.FloatTensor of size 1]
, 
 499.6154
[torch.FloatTensor of size 1]
, 
 497.0607
[torch.FloatTensor of size 1]
]
